In [2]:
%cd D:\Limud\spcup

D:\Limud\spcup


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import glob
import os
import pickle
import scipy
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from mrmr import mrmr_classif
from sklearn.model_selection import GridSearchCV
from sktime.classification.hybrid import HIVECOTEV2
from sklearn.cluster import KMeans
from typing import List
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

!pip install mrmr_selection
!pip install lightgbm


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
def np_load_files(dir_path, file_name):
    files = [np.load(os.path.join(root, file)).T
             for root, dirs, files in os.walk(dir_path) 
             for file in files if(file.endswith(file_name))]
    try:
        files = np.stack(files)
    except ValueError:
        pass
    return files

def pd_load_test_files(dir_path, file_name, transpose = False):
    if transpose:
        
        files = {root[10:]: pd.DataFrame(np.load(os.path.join(root, file)).T)
             for root, dirs, files in os.walk(dir_path) 
             for file in files if(file.endswith(file_name)) if 'sub' in root}
    else:
        files = {root[10:]: pd.DataFrame(np.load(os.path.join(root, file)))
             for root, dirs, files in os.walk(dir_path) 
             for file in files if(file.endswith(file_name)) if 'sub' in root}
    return pd.concat(files, axis=1)

def vector_to_matrix(vector):
    res = []
    for subject in vector:
        n = int(np.sqrt(2 * len(subject))) + 1
        mat = np.zeros((n, n))
        idx = np.triu_indices(n, k=1)  # k=1 skips the main diagonal
        mat[idx] = subject
        res.append(mat + mat.T + np.eye(n))
    return res

def matrix_to_vector(data):
    return np.stack([[val for i, row in enumerate(array) for j, val in enumerate(row) if i < j] for array in data])

def transform_to_log(data):
    transformed_data = []
    mat_data = vector_to_matrix(data)
    for mat in mat_data:
        S, vl = scipy.linalg.eig(mat)
        transformed_data.append( (vl @ np.diag(np.log(S)) @ vl.T).real )
    return matrix_to_vector(transformed_data)

def find_matrix_indices(k, n):
    for i in range(n):
        if i == np.floor((2*k + (i+1)*(i+2))/(2*n)):
            break
    j = int(np.floor((k + (i+1)*(i+2) / 2) % n))
    return i, j

def shuffle_three_arrays(arr1, arr2, arr3):
    import random
    temp = list(zip(arr1, arr2, arr3))
    random.shuffle(temp)
    return zip(*temp)

In [5]:
icn_bp = np_load_files("data/train/bp", 'icn_tc.npy')
fnc_bp = np_load_files("data/train/bp", 'fnc.npy').squeeze()
icn_sz = np_load_files("data/train/sz", 'icn_tc.npy')
fnc_sz = np_load_files("data/train/sz", 'fnc.npy').squeeze()
icn_test = np_load_files("data/test", 'icn_tc.npy')
fnc_test = np_load_files("data/test", 'fnc.npy').squeeze()
fnc_test_df = pd_load_test_files("data/test", 'fnc.npy')
dfnc_train_bp = np_load_files("data/train/bp", 'dFNC.npy')
dfnc_train_sz = np_load_files("data/train/sz", 'dFNC.npy')
dfnc_test = fnc_test = np_load_files("data/test", 'dFNC.npy')
dfnc_test_df = pd_load_test_files("data/test", 'dFNC.npy', True)


In [6]:
# for fnc in fnc_test:
#     print(fnc.shape)
fnc_test = fnc_test_df.to_numpy().T

In [7]:
dfnc_bp = np.concatenate(dfnc_train_bp, axis=1)
dfnc_sz = np.concatenate(dfnc_train_sz, axis=1)
dfnc_test_cat = np.concatenate(dfnc_test,axis=1).T

In [8]:
n = fnc_sz.shape[0] + fnc_bp.shape[0]
fnc_train = np.concatenate([fnc_sz, fnc_bp])
dfnc_train = np.concatenate([dfnc_sz, dfnc_bp], axis=1).T
m = dfnc_train.shape[0]
y_train_dfnc = np.zeros(m)
y_train_dfnc[dfnc_sz.shape[1]: ] = 1 

y_train = np.zeros(n)
y_train[fnc_sz.shape[0]: ] = 1 
icn_train = icn_sz + icn_bp

## shuffle examples
res1, res2, res3 = shuffle_three_arrays(icn_train, fnc_train, y_train)
icn_train, fnc_train, y_train = list(res1), np.stack(res2), np.stack(res3) 

## test
test_subjects = np.stack(fnc_test_df.columns)[:, 0]
test_subjects_dfnc = np.stack(dfnc_test_df.columns)[:, 0]


In [9]:
def test_models(model_list: List[object], X_train, y_train, X_test, subjects_test,
                save_test_predictions: bool = True, test_name: int = 0):
    chosen_model = None
    best_score = 0
    final_scores = []
    for model in model_list:
        scores = []
        k_fold = KFold(n_splits=10, random_state=0, shuffle=True)
        for train_idx, val_idx in k_fold.split(X_train):
            X_train_part, X_val, y_train_part, y_val = X_train[train_idx], X_train[val_idx], \
                                             y_train[train_idx], y_train[val_idx]
            model.fit(X_train_part, y_train_part)
            scores.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1]))
        score = np.mean(scores)
        print(f'model {model} average accuracy is = {score * 100}%')
        if best_score < score:
            chosen_model = model
            best_score = score
        if save_test_predictions:
            model.fit(X_train, y_train)
            test_prediction = model.predict_proba(X_test)[:, 1]
            results = pd.DataFrame()
            results['ID'] = subjects_test
            results['Predicted'] = test_prediction
            results = results.groupby(['ID']).mean()

            # Save results:
            file_name = 'results.csv'
            results_file = f'results/{model.__class__.__name__}_{test_name}_{round(score, 2)}_{file_name}'
            results.to_csv(results_file)
        final_scores.append(test_prediction)
    print(f'Best model is: {chosen_model}')
    return chosen_model, final_scores

In [10]:
mlp_params = {
    'hidden_layer_sizes': [(100,), (10,), (100, 10,), ],
    'activation': ['tanh', 'relu'],
    'alpha': [5e-2, 1e-3, 1e-4],}

def create_models(sec_model_list: bool = True):
    lgbm_model = LGBMClassifier(n_estimators=1000, random_state=0, max_depth=4, num_leaves=18)
    optim_mlp_model = GridSearchCV(MLPClassifier(batch_size=500), mlp_params, n_jobs=-1)
    mlp_model1 = MLPClassifier(activation='tanh', hidden_layer_sizes = (100, ), batch_size=500, alpha=0.05)
    svm_model = GridSearchCV(SVC(probability=True, class_weight='balanced'), {'kernel': ['rbf', 'linear', 'poly'], 'C': [0.2, 0.5, 0.8, 1, 1.2, 1.5, 2]})
    svm_model_rbf = SVC(probability=True, class_weight='balanced', kernel ='rbf', C=1.0)
    rfc_model = RandomForestClassifier(n_estimators=100)
    knn_model = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': [3,5, 7,9, 11, 13]}, n_jobs=-1)
    xgboost_model = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0, max_depth=4)
    bagging_model = BaggingClassifier(SVC(probability=True, class_weight='balanced', C =1.0), n_estimators=3, random_state=10)
    return  [svm_model_rbf, mlp_model1, xgboost_model]


In [ ]:
## test with original data
models = create_models()
best_model = test_models(models, dfnc_train, y_train_dfnc, dfnc_test_cat, test_subjects_dfnc, save_test_predictions=True)

In [ ]:
# test with mrmr data with selected features (19)
return_scores = True
selected_features = mrmr_classif(X=pd.DataFrame(fnc_train), y=y_train, K=19, return_scores=return_scores)
if return_scores:
    full_features = selected_features[1]
    selected_features = selected_features[0]
X_train_mrmr = fnc_train[:, selected_features]
X_test_mrmr = fnc_test[:, selected_features]
models = create_models()
best_model, res = test_models(models, X_train_mrmr, y_train, X_test_mrmr, test_subjects, save_test_predictions=True, test_name=19_17_03_2023)


In [57]:
models[1].best_params_

{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (100,)}

In [11]:
from scipy.linalg import sqrtm, logm, expm, norm, inv

def symmetric_matrix(A):
    return (A + A.T) / 2


def riemannian_mean(dFNC):
    num_matrices = dFNC.shape[2]
    if num_matrices == 1:
        return dFNC
    M = np.mean(dFNC, axis=2)
    weights = (np.ones((num_matrices, 1)) / num_matrices)
    for iteration in range(30):
        A = sqrtm(M)
        B = inv(A)
        S = np.zeros_like(M)
        for j in range(num_matrices):
            current_matrix = dFNC[:, :, j]
            if scipy.linalg.eig(current_matrix)[0].min() <= 0:
                continue
            BCB = symmetric_matrix(B @ current_matrix @ B)
            S += weights[j] * logm(BCB).real
        M = symmetric_matrix(A @ expm(S) @ A)
        if norm(S, 'fro') < 1e-6:
            break
    return M.real


def conjection(e_dict, labels, fnc_data):
    rei_data = []
    fnc_mat_data = np.stack(vector_to_matrix(fnc_data))
    for label, fnc in zip(labels, fnc_mat_data):
        E = e_dict[label]
        fnc_hat = E @ fnc @ E.T
        rei_data.append(fnc_hat)
    fnc_hat = np.stack(rei_data)
    fnc_hat = np.stack(matrix_to_vector(fnc_hat.real))
    return fnc_hat

def build_E(full_dict):
    e_dict = {}
    for key, data in full_dict.items():
        print(f'Key={key}')
        mat_data = np.stack(vector_to_matrix(data))
        x_bar = riemannian_mean(mat_data.transpose(1,2,0)).squeeze()
        e_dict[key] = scipy.linalg.fractional_matrix_power(x_bar, -0.5)
    return e_dict

In [12]:
from collections import OrderedDict

def create_fnc_diff(labels, fncs):
    diff_len = OrderedDict()
    for label, fnc in zip(labels, fncs):
        if label in diff_len:
            diff_len[label].append(fnc)
        else:
            diff_len[label] = [fnc]
    return diff_len


# kmeans = KMeans(n_clusters=6, random_state=100, init='k-means++').fit(np.concatenate([fnc_train_log,fnc_test_log]))
full_data_dict = create_fnc_diff([icn.shape[1] for icn in icn_train + icn_test], np.concatenate([fnc_train, fnc_test]))
group_sizes = {key: len(val) for key, val in full_data_dict.items()}
print(f'The size of each group: {group_sizes}')

The size of each group: {234: 65, 230: 364, 204: 244, 100: 8, 134: 95, 210: 9, 208: 1}


In [55]:
e_dict = build_E(full_data_dict)

Key=230
Key=204
Key=134
Key=234
Key=100
Key=210
Key=208


In [56]:
X_train_riemann = conjection(e_dict, [icn.shape[1] for icn in icn_train], fnc_train)
X_test_riemann = conjection(e_dict, [icn.shape[1] for icn in icn_test], fnc_test)

In [ ]:
## test with riemann full data
models = create_models()
best_model, scores = test_models(models, dfnc_train, y_train_dfnc, dfnc_test, test_subjects_dfnc, save_test_predictions=True, test_name='full')

In [ ]:
dfnc_test_cat.shape

In [ ]:
return_scores = True
selected_features_rei = mrmr_classif(X=pd.DataFrame(dfnc_train), y=y_train_dfnc, K=19, return_scores=return_scores)
if return_scores:
    full_features = selected_features_rei[1]
    selected_features_rei = selected_features_rei[0]
X_train_riemann_mrmr = dfnc_train[:, selected_features_rei]
X_test_riemann_mrmr = dfnc_test_cat[:, selected_features_rei]

models = create_models()
best_model, res = test_models(models, X_train_riemann_mrmr, y_train_dfnc, X_test_riemann_mrmr, test_subjects_dfnc, save_test_predictions=True, test_name=f'mrmr{19}={21_03_2023}_right_order')

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [04:51<00:00, 15.35s/it]


In [ ]:
plt.plot(sorted(full_features, reverse=True))
plt.title('MRMR features relavence')
plt.show()
full_features = np.stack(full_features)
len(full_features[full_features>5])

In [47]:
from sklearn.manifold import TSNE
import umap


X_train_riemann_embed = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(fnc_train)
X_test_riemann_embed = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(fnc_test)

# X_train_riemann_embed = umap.UMAP().fit_transform(X_train_riemann_mrmr)
# X_test_riemann_embed = umap.UMAP().fit_transform(X_test_riemann_mrmr)


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
df = pd.DataFrame()
df["y"] = y_train
df["x1"] = X_train_riemann_embed[:,0]
df["x2"] = X_train_riemann_embed[:,1]

sns.scatterplot(x="x1", y="y", data=df) 
plt.show()

df = pd.DataFrame()
df["y"] = np.zeros(315)
df["x1"] = X_test_riemann_embed[:,0]
df["x2"] = X_test_riemann_embed[:,1]

sns.scatterplot(x="x1", y="x2", hue=df.y.tolist(),palette=sns.color_palette("hls",1), data=df) 
plt.show()
